In [1]:
# Import libraries.
import matplotlib.pyplot as plt
import networkx as nx
import random

In [2]:
# Define constants.
NUMBER_OF_NODES = 100
EDGE_PROBABILITY = 0.1

In [3]:
# Read names from file.
text_file = open('names.txt', 'r')
lines = text_file.readlines()
text_file.close()

In [4]:
# Put names in list and create set to store used ones.
names = [line.split()[0] for line in lines]
used_names = set()

In [5]:
# # TEST: Test names.
# names = ['orange','yellow','blue','green','grey']
# used_names = ['orange','yellow','blue','green','grey']

In [6]:
# Generate random networks.
network_1 = nx.fast_gnp_random_graph(NUMBER_OF_NODES, EDGE_PROBABILITY, seed = 1, directed = True)
network_2 = nx.fast_gnp_random_graph(NUMBER_OF_NODES, EDGE_PROBABILITY, seed = 2, directed = True)
network_3 = nx.fast_gnp_random_graph(NUMBER_OF_NODES, EDGE_PROBABILITY, seed = 3, directed = True)

In [7]:
# # TEST: Test networks.
# network_1 = nx.DiGraph()
# network_1.add_edges_from([('n1_1', 'n1_2'),('n1_1', 'n1_3'), ('n1_2', 'n1_3')])
# network_2 = nx.DiGraph()
# network_2.add_edges_from([('n2_1', 'n2_3'),('n2_2', 'n2_1'), ('n2_3', 'n2_2')])
# network_3 = nx.DiGraph()
# network_3.add_edges_from([('n3_1', 'n3_2'),('n3_2', 'n3_3'), ('n3_2', 'n3_4'), ('n3_3', 'n3_4')])

In [8]:
# Put networks in a list.
networks = [network_1, network_2, network_3]

In [9]:
# Relabel network nodes.
for network in networks:
    relabel_dict = {}
    for i in range(NUMBER_OF_NODES):
        relabel_dict[i] = 'n' + str(networks.index(network) + 1) + '_'  + str(i)
    nx.relabel_nodes(network, relabel_dict, copy = False)

In [10]:
# Add network node and edge attributes.
for network in networks:
    for (u, v) in network.edges():
        network.edge[u][v]['weight'] = random.random()    
    
    names_temp = names.copy()
    name_dict = {}
    threshold_dict = {}
    
    for node in network.nodes_iter():
        name = random.choice(names_temp)
        name_dict[node] = name
        used_names.add(name)
        names_temp.remove(name)
        threshold_dict[node] = random.random()
    
    nx.set_node_attributes(network, 'name', name_dict)
    nx.set_node_attributes(network, 'threshold', threshold_dict)
    
    # Normalize edge weights so that all incoming edge will add up to 1 per node.
    for node in network.nodes_iter():
        in_edge_sum = 0
        in_edges = network.in_edges(node)
        for edge in in_edges:
            in_edge_sum += network[edge[0]][edge[1]]['weight']
            
        for edge in in_edges:
            network[edge[0]][edge[1]]['weight'] /= in_edge_sum

In [11]:
# # TEST: Add node names for test data.
# network_1.node['n1_1']['name'] = 'orange'
# network_1.node['n1_2']['name'] = 'yellow'
# network_1.node['n1_3']['name'] = 'blue'

# network_2.node['n2_1']['name'] = 'green'
# network_2.node['n2_2']['name'] = 'yellow'
# network_2.node['n2_3']['name'] = 'blue'

# network_3.node['n3_1']['name'] = 'orange'
# network_3.node['n3_2']['name'] = 'grey'
# network_3.node['n3_3']['name'] = 'green'
# network_3.node['n3_4']['name'] = 'yellow'

In [12]:
# # Print for check.
# print('################# NETWORK 1 #################')
# print(network_1.nodes())
# print(network_1.edges())
# print(nx.get_edge_attributes(network_1, 'weight'))
# print(nx.get_node_attributes(network_1, 'name'))
# print(nx.get_node_attributes(network_1, 'threshold'))

# print('################# NETWORK 2 #################')
# print(network_2.nodes())
# print(network_2.edges())
# print(nx.get_edge_attributes(network_2, 'weight'))
# print(nx.get_node_attributes(network_2, 'name'))
# print(nx.get_node_attributes(network_2, 'threshold'))

# print('################# NETWORK 3 #################')
# print(network_3.nodes())
# print(network_3.edges())
# print(nx.get_edge_attributes(network_3, 'weight'))
# print(nx.get_node_attributes(network_3, 'name'))
# print(nx.get_node_attributes(network_3, 'threshold'))

In [13]:
# Combine networks.
combined_network = nx.compose_all(networks)

In [14]:
# Add new edges with gateway nodes with weight.
for name in used_names:
    for network in networks:
        for node, node_name in nx.get_node_attributes(network, 'name').items():
            if name == node_name:
                out_edges = []
                if len(network.out_edges(node)) > 0:
                    out_edges = network.out_edges(node)
                for edge in out_edges:
                    combined_network.remove_edge(edge[0],edge[1])
                    combined_network.add_edge(name, edge[1], attr_dict = {'weight': network.edge[edge[0]][edge[1]]['weight']})

In [15]:
# Add 0 threshold to gateway nodes.
for name in used_names:
    if name in combined_network:
        combined_network.node[name]['threshold'] = 1

In [16]:
# Add connections for same user in different networks.
# Representative nodes - gateway nodes connection.
for name in used_names:
    if name in combined_network:
        for node, node_name in nx.get_node_attributes(combined_network, 'name').items():
            if name == node_name:
                combined_network.add_edge(name, node, attr_dict = {'weight': combined_network.node[node]['threshold']})
                combined_network.add_edge(node, name, attr_dict = {'weight': 1})
                
# Representative nodes - representative nodes connection.
for name in used_names:
    for node, node_name in nx.get_node_attributes(combined_network, 'name').items():
        for node2, node_name2 in nx.get_node_attributes(combined_network, 'name').items():
            if node != node2 and node_name == node_name2 and not combined_network.has_edge(node, node2):
                combined_network.add_edge(node, node2, attr_dict = {'weight': combined_network.node[node2]['threshold']})
                combined_network.add_edge(node2, node, attr_dict = {'weight': combined_network.node[node]['threshold']})

In [17]:
# Add dummy nodes.
for name in used_names:
    is_in_network = ['False0', 'False1', 'False2']
    for network in networks:
        for node, node_name in nx.get_node_attributes(network, 'name').items():
            if name == node_name:
                is_in_network[networks.index(network)] = 'True' + str(networks.index(network))
                   
    for found in is_in_network:
        index = is_in_network.index(found)
        if ('False' + str(index)) == found:
            networks[index].add_node('n' + str(index + 1) + '_' + str(networks[index].number_of_nodes() + 1), attr_dict = {'name': name})
            combined_network.add_node('n' + str(index + 1) + '_' + str(networks[index].number_of_nodes()), attr_dict = {'name': name, 'threshold': 1})

In [18]:
# # Print for check.
# print(combined_network.number_of_nodes())
# print(combined_network.number_of_edges())
# print(combined_network.nodes())
# print(combined_network.edges())